# Helper functions

The function `convertBasisE10(vE, E, N)` converts a number expressed in base-`E` (and represented as the vector `vE`) into its decimal representation `p` (as an integer).

*Convention:* `vE(1)` represents the idiosyncratic  state in the present period and `vE(N)` represents the past idosyncratic state N-1 periods

In [3]:
function convertBasisE10(vE::Vector{I}, E::I)::I where {I<:Integer}
# converts the a vector vE into an integer p 

    @assert (minimum(vE)≥1)&&(maximum(vE)≤E)
    
    N = length(vE)    
    p = one(I) #otherwise p lieas between 0 and (E+1)^N - 1
    for k ∈ eachindex(vE)
        p += (vE[k]-1)*E^(N-k)
    end
    return p
end

convertBasisE10 (generic function with 1 method)

The function `convertBasis10E(p, E, N)` converts a number expressed in base-10 (and represented as the integer p) into its base-`E` representation `vE` (as a vector of length `N`). This is the inverse of `convertBasisE10`.

*Convention:* `vE(1)` represents the idiosyncratic  state in the present period and `vE(N)` represents the past idosyncratic state N-1 periods

In [4]:
function  convertBasis10E(p::I, E::I, N::I)::Vector{I} where{I<:Integer}

    vE = zeros(I, N)    
    @assert (p≥1)&&(p≤E^N)
    ptemp = p-1;
    for k ∈ eachindex(vE)
        ptemp, r = divrem(ptemp,E)
        vE[N-k+1] = 1+r        
    end
    return vE
end

convertBasis10E (generic function with 1 method)

In [8]:
function myMultSp(M::SparseMatrixCSC{T,I}, V::SparseVector{T,I}, start_i::I, end_i::I, start_j::I, end_j::I)  where{I<:Integer, T<:Real}
# Computes the vector U, such that:
    # U[start_i:end_i] = M[start_i:end_i:,start_j:end_j] * V[start_j:end_j] and U[i] = 0 if i∉start_j:end_j
    
    (rowsM, colsM, valsM) = findnz(M)
    (colsV, valsV) = findnz(V)
    res_iter = start_i:end_i
    res_vals = zeros(T, end_i-start_i+1)
    for (iM,jM,vM) ∈ zip(rowsM, colsM, valsM)
        if (iM ≥ start_i)&&(iM ≤ end_i)&&(jM ≥ start_j)&&(jM ≤ end_j)
            jVs = findall(x->x==jM,colsV)
            if !(isempty(jVs))                
                res_vals[iM-start_i+1] += vM * valsV[jVs[1]]
            end
        end            
    end
    
    n = sum(res_vals.>1e-16)
    toR_iter = zeros(I, n)
    toR_vals = zeros(T, n)
    count = 0
    for (i, v) in enumerate(res_vals)
        if v > 1e-16
            count +=1
            toR_iter[count] = res_iter[i]
            toR_vals[count] = v
        end
    end
    toR = sparsevec(toR_iter, toR_vals, length(V))
#     @show findnz(toR)
    return (toR_iter, toR_vals, length(V))
end


5-element SparseVector{Float64, Int64} with 1 stored entry:
  [2]  =  2.0

In [12]:
#Constructs a sparse vector of the size of truncated history
#The indices of the sparse vector (i.e. of non zero size) provides the list of non-empty histories
function historySizes(N::I, Πy::Matrix{T};
        maxiter::I=1000000, tol::T=1e-16) where{I<:Integer, T<:Real}
    ny = size(Πy,1)
    
    _, Sy = powm!(Πy', ones(typeof(Πy[1]), ny),maxiter = maxiter,tol = tol)
    Sy /= sum(Sy)
    Ntot = ny^N #total nb of histories
    Sh   = zeros(typeof(Πy[1]), Ntot) # size of truncated histories
    y0h   = zeros(I, Ntot) # current productivity indices of history
    for i ∈ eachindex(Sh)
        vE =  convertBasis10E(i, ny, N)
        Sh[i] = Sy[vE[N]] #distribution according to the terminal productity level
        y0h[i] = vE[1]
        for j = N-1:-1:1
            Sh[i] *= Πy[vE[j+1],vE[j]] 
            # We move backwards and compute the probability to move from productivity 
            # vE[j] to productivity vE[j+1]
        end  
    end
    ind_h, S_h = findnz(sparsevec(Sh))
    return ind_h, S_h, y0h[ind_h]
end


# Computes a matrix ((na \times ny), ny^N)of the distribution of histories over the 
# product grid asset \times productivity. More precisely, if: 
# statDist_h = historyDist(ny, N, stationaryDist, transitMat)
# Then statDist_h[:,h] is the distribution of history h over the grid asset,productivity

function historyDist(ny::I, N::I, stationaryDist::Matrix{T}, 
            transitMat::SparseMatrixCSC{T,I})::Matrix{T} where{I<:Integer, T<:Real}
    
    na = div(size(transitMat,1),ny)
    Nh = ny^N
    statDist_h = zeros(T, length(stationaryDist), Nh)
    temp_v = zeros(T, na)
    for i ∈ 1:Nh
        vE     = convertBasis10E(i, ny, N)
        shift0 = (vE[N]-1)*na
        statDist_h[1+shift0:na+shift0,i] =  stationaryDist[:,vE[N]]
        for j=N-1:-1:1                
            shift1 = (vE[j+1]-1)*na
            shift0 = (vE[j]-1)*na
            temp_v .= statDist_h[1+shift1:na+shift1,i]
            statDist_h[:,i] .= zero(T)
            statDist_h[1+shift0:na+shift0,i] .= (
                transitMat[1+shift0:na+shift0,1+shift1:na+shift1] * temp_v)
        end
    end
    return statDist_h
end

function historyTrans(N::I,ind_h::Vector{I},Πy::Matrix{T}) where{I<:Integer,T<:Real}
    ny = length(Πy[:,1])
    Ntot = length(ind_h)
    Πh = spzeros(T,Ntot,Ntot)
    for (i,h) ∈ enumerate(ind_h)
        vE_h = convertBasis10E(h, ny, N)
        for (j,ht) ∈ enumerate(ind_h)
            vE_ht = convertBasis10E(ht, ny, N)
            if all(vE_ht[2:end] .== vE_h[1:end-1])
                Πh[i,j] = Πy[vE_h[1],vE_ht[1]]
            end
        end
    end
#     @show "here"
    return Πh
end

function credit_constrained_h(shareCC::T, S_h::Vector{T}, c_h::Vector{T};method::String="first larger") where{T<:Real}
#     S_cc = sum(stationaryDist[1,:]) #Share of credit constrained agents
#     c_h  = sum(statDist_h .* repeat(gc[:],1,Ntot), dims=1)[:]./S_h
    ind_c_indiv = sortperm(c_h)
    S_h_sorted = S_h[ind_c_indiv]
    if method=="closest"
        i_cc = argmin(abs.(cumsum(S_h_sorted) .- shareCC)) #closest distance
    else #(method=="first larger")
        i_cc = findfirst(x->x>0.0, cumsum(S_h_sorted) .- shareCC)#at least as large
    end
    return i_cc, ind_c_indiv[1:i_cc]
end

function Project_plan(N::Integer, # length of the truncation
                      solution::AiyagariSolution, 
                      params::Params;maxiter=1000000,tol=1e-16)
    @unpack β,α,δ,tk,tl,u,u′,u′′,v,v′,l_supply,na,a_min,aGrid,ny,ys,Πy = params
    @unpack ga,gc,gl,R,w, A,K,L,transitMat,stationaryDist,residEuler = solution
    (ind_h, S_h, ind_y0_h) = historySizes(N, Πy)
    y0_h = [ys[i] for i in ind_y0_h]
    T = typeof(Πy[1,1])
    statDist_h = historyDist(ny, N, stationaryDist, transitMat)[:,ind_h]
    Ntot = length(ind_h)#nb of history with positive size
    Π_h = historyTrans(N,ind_h,Πy)
    
    #Compute allocations
    c_h       = sum(statDist_h .* repeat(gc[:],1,Ntot), dims=1)[:]./S_h   
    l_h       = sum(statDist_h .* repeat(gl[:],1,Ntot), dims=1)[:]./S_h
    a_beg_h   = sum(statDist_h .* repeat(aGrid[:],ny,Ntot), dims=1)[:]./S_h
    a_end_h   = sum(statDist_h .* repeat(ga[:],1,Ntot), dims=1)[:]./S_h
    u_h       = sum(statDist_h .* repeat(u.(gc[:]),1,Ntot), dims=1)[:]./S_h
    u′_h      = sum(statDist_h .* repeat(u′.(gc[:]),1,Ntot), dims=1)[:]./S_h
    u′′_h     = sum(statDist_h .* repeat(u′′.(gc[:]),1,Ntot), dims=1)[:]./S_h
    v_h       = sum(statDist_h .* repeat(v.(gl[:]),1,Ntot), dims=1)[:]./S_h
    v′_h      = sum(statDist_h .* repeat(v′.(gl[:]),1,Ntot), dims=1)[:]./S_h
    # ly_h  = sum(statDist_h .* repeat(((repeat(ys',na,1).*gl)[:]).^τ,1,Ntot), dims=1)[:]./S_h
    ly_h  = sum(statDist_h .* repeat(((repeat(ys',na,1).*gl)[:]),1,Ntot), dims=1)[:]./S_h
    resid_E_h = sum(statDist_h .* repeat(residEuler[:],1,Ntot), dims=1)[:]./S_h
    
    #Define credit constrained histories
    share_cc = sum(stationaryDist[1,:]) #Share of credit constrained agents
    nb_cc_h, ind_cc_h = credit_constrained_h(share_cc, S_h, c_h;method="first larger")
    
    allocation_proj = Allocation_proj(
        S_h=S_h,
        Π_h=Π_h,
        y0_h=y0_h,
        a_beg_h=a_beg_h,
        a_end_h=a_end_h,
        c_h=c_h,
        l_h=l_h,
        ly_h=ly_h,
        u_h=u_h,
        u′_h=u′_h,
        u′′_h=u′′_h,
        v_h=v_h,
        v′_h=v′_h,
        resid_E_h=resid_E_h,
        nb_cc_h=nb_cc_h,
        ind_cc_h=ind_cc_h)
    
    ξu0 = u_h./u.(c_h)
    ξu1 = u′_h./u′.(c_h)
    ξu2 = u′′_h./u′′.(c_h)


    # ξuE = ((I-β*R*Π_h)\resid_E_h)./u′.(c_h)
    
    # ξuE = (((I-β*R*Π_h')*spdiagm(u′.(c_h)))\resid_E_h)

    # ξuE = (((I-β*R*Π_h')*spdiagm(u′.(c_h)))\1)


    ξuE = ((I-β*R*Π_h)\resid_E_h)./u′.(c_h)


    # ξuE = (((I-β*R*Π_h')*spdiagm(u′.(c_h))))

     

    

    # (I-β*R*Π_h)*ξuE*u′.(c_h) = resid_E_h
    # ((I-β*R*Π_h*ξuE*u′.(c_h) = resid_E_h
    # ξuE*u′.(c_h) - β*R*Π_h*ξuE*u′.(c_h) = resid_E_h
    ξy  = ly_h./(y0_h.*l_h)
    

    ξv0 = v_h./v.(l_h)
    # ξv1 = τ*w*ly_tau_h.*u′_h./(l_h.*v′.(l_h)) #(equivalent to τ*w*ξy*.(y0_h.*l_h).^τ?*ξu1.*u′.(c_h)./l_h)
    #ξv1 = w*ly_h.*u′_h./(l_h.*v′.(l_h)) #(equivalent to τ*w*ξy*.(y0_h.*l_h).^τ?*ξu1.*u′.(c_h)./l_h)

   # ξv1 = w*y0_h.*ξu1 .*(u′.(c_h) ./(v′.(l_h))) #(equivalent to τ*w*ξy*.(y0_h.*l_h).^τ?*ξu1.*u′.(c_h)./l_h)
    #ξv1 = w*y0_h.*(ξu1_h .*u′.(c_h)) ./(v′.(l_h)) #(equivalent to τ*w*ξy*.(y0_h.*l_h).^τ?*ξu1.*u′.(c_h)./l_h)

    ξv1 = w*y0_h.*u′_h./(v′.(l_h)) #(equivalent to τ*w*ξy*.(y0_h.*l_h).^τ?*ξu1.*u′.(c_h)./l_h)



    ξs = ξs_struct(
        ξu0=ξu0,
        ξu1=ξu1,
        ξu2=ξu2,
        ξuE=ξuE,
        ξy=ξy,
        ξv0=ξv0,
        ξv1=ξv1)
    
    return Projection(
        N = N,
        Ntot = Ntot,
        ind_h=ind_h,
        allocation_proj = allocation_proj,
        ξs = ξs)

end


function test(N::Integer, # length of the truncation
                      solution::AiyagariSolution, 
                      params::Params;maxiter=1000000,tol=1e-16)
    @unpack β,α,δ,tk,tl,u,u′,u′′,v,v′,l_supply,na,a_min,aGrid,ny,ys,Πy = params
    @unpack ga,gc,gl,R,w, A,K,L,transitMat,stationaryDist,residEuler = solution
    (ind_h, S_h, ind_y0_h) = historySizes(N, Πy)
    y0_h = [ys[i] for i in ind_y0_h]
    T = typeof(Πy[1,1])
    statDist_h = historyDist(ny, N, stationaryDist, transitMat)[:,ind_h]
    Ntot = length(ind_h)#nb of history with positive size
    Π_h = historyTrans(N,ind_h,Πy)
    
    #Compute allocations
    c_h       = sum(statDist_h .* repeat(gc[:],1,Ntot), dims=1)[:]./S_h   
    l_h       = sum(statDist_h .* repeat(gl[:],1,Ntot), dims=1)[:]./S_h
    a_beg_h   = sum(statDist_h .* repeat(aGrid[:],ny,Ntot), dims=1)[:]./S_h
    a_end_h   = sum(statDist_h .* repeat(ga[:],1,Ntot), dims=1)[:]./S_h
    u_h       = sum(statDist_h .* repeat(u.(gc[:]),1,Ntot), dims=1)[:]./S_h
    u′_h      = sum(statDist_h .* repeat(u′.(gc[:]),1,Ntot), dims=1)[:]./S_h
    u′′_h     = sum(statDist_h .* repeat(u′′.(gc[:]),1,Ntot), dims=1)[:]./S_h
    v_h       = sum(statDist_h .* repeat(v.(gl[:]),1,Ntot), dims=1)[:]./S_h
    v′_h      = sum(statDist_h .* repeat(v′.(gl[:]),1,Ntot), dims=1)[:]./S_h
    # ly_h  = sum(statDist_h .* repeat(((repeat(ys',na,1).*gl)[:]).^τ,1,Ntot), dims=1)[:]./S_h
    ly_h  = sum(statDist_h .* repeat(((repeat(ys',na,1).*gl)[:]),1,Ntot), dims=1)[:]./S_h
    resid_E_h = sum(statDist_h .* repeat(residEuler[:],1,Ntot), dims=1)[:]./S_h
    
    #Define credit constrained histories
    share_cc = sum(stationaryDist[1,:]) #Share of credit constrained agents
    nb_cc_h, ind_cc_h = credit_constrained_h(share_cc, S_h, c_h;method="first larger")
    
    allocation_proj = Allocation_proj(
        S_h=S_h,
        Π_h=Π_h,
        y0_h=y0_h,
        a_beg_h=a_beg_h,
        a_end_h=a_end_h,
        c_h=c_h,
        l_h=l_h,
        ly_h=ly_h,
        u_h=u_h,
        u′_h=u′_h,
        u′′_h=u′′_h,
        v_h=v_h,
        v′_h=v′_h,
        resid_E_h=resid_E_h,
        nb_cc_h=nb_cc_h,
        ind_cc_h=ind_cc_h)
    
 
    
    return resid_E_h

end





UndefVarError: UndefVarError: SparseMatrixCSC not defined